In [2]:
import os
import sys
import pandas as pd
from tqdm import tqdm
import re
import unidecode
from datetime import datetime
import seaborn as sns
from matplotlib import pyplot as plt

In [3]:
tqdm.pandas() 

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [6]:
from src.data import make_dataset

## Configuracion

In [17]:
DATA_PATH_CLEAN = '../../data/interim/'
DATA_PATH_OUTPUT = '../../data/final/'

file_name_tramites_clean = DATA_PATH_CLEAN+'tramites_fraudes/tramites_clean_{}.parquet'
file_name_consumo_clean = DATA_PATH_CLEAN+'facturacion/consumo_clean_{}.parquet'
file_name_notas_clean = DATA_PATH_CLEAN+'notas/notas_clean_{}.parquet'
file_name_hist_puntos_clean = DATA_PATH_CLEAN+'puntos/hist_puntos_clean_{}.parquet'


config_caidas = [(1,4,90),(1,3,90),(2,3,90),(1,6,90),(3,3,90),(6,5,10),(6,6,10),(6,4,10),(6,1,10),(5,5,10)]
config_constantes = [8,9,10,3,4,5]
cols_hist_ptos = ['localizacion','fecha','estado_medidor','estado_contrato','id_tarifa']
config_below_g =[(['control_manzana'],DATA_PATH_CLEAN+'/stats/stat_control_manzana_{}') ,
                 (['id_distrito','id_tarifa'],DATA_PATH_CLEAN+'/stats/stat_distrito_tarifa_{}') ,
                 (['id_distrito','id_cant_fases'],DATA_PATH_CLEAN+'/stats/stat_distrito_fases_{}') ]
var_puntos = ['localizacion','id_provincia','id_canton','id_distrito','id_tipo_medidor','tipo_conexion','area_peligrosa',
               'id_tarifa','id_estado_contrato','tip_instalacion','id_cant_fases','id_ciu','marca_medidor',
               'nic', 'tipo_cliente', 'medidor','ind_fraude', 'estado_medidor','id_clase','clasif_contrato','nise'
                ,'fecha_instalacion']
file_name_traindata = DATA_PATH_CLEAN+'/traindatasets/train_data_{}.parquet'
file_name_traindata_final = DATA_PATH_OUTPUT+'/train_data_{}.parquet'
cant_periodos = 12
fecha_fraud_list = ['2023-07-01','2023-08-01','2023-09-01']

## Prepare data sources 
---

In [18]:
lista_anios = make_dataset.get_start_end_years_and_years_range(fecha_fraud_list,cant_periodos)

In [9]:
df_ordenes = make_dataset.load_data(file_name_tramites_clean,lista_anios)
df_consumo = make_dataset.load_data(file_name_consumo_clean,lista_anios)
df_notas = make_dataset.load_data(file_name_notas_clean,lista_anios)
df_hist_puntos = make_dataset.load_data(file_name_hist_puntos_clean,lista_anios,cols_hist_ptos)
df_static_puntos = make_dataset.load_data(file_name_hist_puntos_clean,[2023])

100%|██████████| 1/1 [00:10<00:00, 10.23s/it]


In [10]:
%%time
df_ordenes,df_consumo,df_notas,df_hist_puntos,df_static_puntos = make_dataset.\
                                                                        prepare_data(df_ordenes,df_consumo,df_notas,df_hist_puntos,df_static_puntos)

CPU times: user 46.4 s, sys: 9.2 s, total: 55.6 s
Wall time: 55.5 s


In [11]:
%%time
make_dataset.prepare_stat_consumo_below_group(fecha_fraud_list,config_below_g,df_consumo,cant_periodos)

100%|██████████| 3/3 [00:56<00:00, 18.81s/it]

CPU times: user 4min 35s, sys: 25.7 s, total: 5min 1s
Wall time: 5min 1s


## Create Train Data Sets

In [12]:
%%time
make_dataset.create_train_data(fecha_fraud_list,df_consumo,df_ordenes,df_hist_puntos,df_notas,df_static_puntos,
                                cant_periodos,config_below_g,config_caidas,config_constantes,
                               var_puntos,file_name_traindata, file_name_traindata_final)

100%|██████████| 3/3 [00:00<00:00, 82.05it/s]

ENTRO PARA CREAR
ENTRI a CREAR TSFEL
*** Feature extraction started ***



/root/aux/cnfl/src/data/make_dataset.py:22: UserWarning: Using default sampling frequency set in configuration file.
  df_result = tsfel.time_series_features_extractor(cfg, df[cols].values.tolist(),verbose=1,n_jobs=-1)



*** Feature extraction finished ***
df_result index: (1388,)
*** Feature extraction started ***


/root/aux/cnfl/src/data/make_dataset.py:22: UserWarning: Using default sampling frequency set in configuration file.
  df_result = tsfel.time_series_features_extractor(cfg, df[cols].values.tolist(),verbose=1,n_jobs=-1)



*** Feature extraction finished ***
df_result index: (1388,)
*** Feature extraction started ***


/root/aux/cnfl/src/data/make_dataset.py:22: UserWarning: Using default sampling frequency set in configuration file.
  df_result = tsfel.time_series_features_extractor(cfg, df[cols].values.tolist(),verbose=1,n_jobs=-1)



*** Feature extraction finished ***
df_result index: (1388,)
temp_vars 14
stat_vars 40
spec_vars 86
df_tsfel (1388, 141)
CPU times: user 48.4 s, sys: 11.3 s, total: 59.8 s
Wall time: 1min 17s
